In [ ]:
import nltk
import ast
import re
import copy
import math
import openpyxl
from collections import Counter
import pandas as pd
import numpy as np
import json
import tensorflow as tf
import tensorflow_hub as hub
import csv
import random
import sklearn
import torch
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from tensorflow import keras
from tensorflow.keras import layers
from sentence_transformers import SentenceTransformer
from transformers import XLMRobertaModel, XLMRobertaTokenizer, BertModel, BertTokenizer, XLMModel, XLMTokenizer, DistilBertModel, DistilBertTokenizer
import sentencepiece
import ipywidgets

# model = XLMRobertaModel.from_pretrained('xlm-roberta-base')
# tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
#
# model = BertModel.from_pretrained('bert-base-multilingual-cased')
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')



In [ ]:
request, intent = [], []  # train set
def input(file):
    data = json.load(open(file))['intents']
    for each in data:
        label = each['name']
        for req in each['samples']:
            r = req['utterance']
            request.append(r)
            intent.append(label)
input('/Users/sumanth.gurram/Desktop/dummy_train_data')
u2l = {}
u2v = {}
file = open('/Users/sumanth.gurram/Desktop/dummy_test_data')
Lines = file.readlines()
Lines = Lines[1:]
it, rq = [], []
for line in Lines:
    split = line.strip().split('\t')
    s = split[2]
    it.append(tent)
    rq.append(split[1])
intent, request = sklearn.utils.shuffle(intent, request)
it, rq = sklearn.utils.shuffle(it, rq)

# print(len(request))
# print(len(rq))
# print(Counter(intent))
# print(Counter(it))
# print(len(Counter(intent)))
# print(len(Counter(it)))

In [ ]:
vector, atts, labId = [], [], []
vec, att, lab = [], [], []
vector, labId = [], [] #train set
vec, lab = [], []      #test set

def pooling(attention_mask, token_embeddings):
    output_vectors = []
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = input_mask_expanded.sum(1)
    sum_mask = torch.clamp(sum_mask, min=1e-9)
    output_vectors.append(sum_embeddings / sum_mask)
    output_vector = torch.cat(output_vectors, 1)
    return output_vector

def xlm_encode(utts):
    tokens = tokenizer.batch_encode_plus(utts, pad_to_max_length=True, max_length = 60, truncation = True)
    tk = torch.tensor(tokens["input_ids"])
    mask = torch.tensor(tokens["attention_mask"])
    print(mask)
    emb = model(tk)
    print(emb[0].shape)
    return emb[0], mask

vectors, train_mask = xlm_encode(request)
vec, test_mask = xlm_encode(rq)

def ang(x,y):
    nx = np.linalg.norm(x)
    ny = np.linalg.norm(y)
    cos = np.dot(x, y)/(nx * ny)
    if cos > 1:
        cos = 1
    elif cos < -1:
        cos = -1
    return 1 - np.arccos(cos)/np.pi

train_embeddings = train_mask.unsqueeze(2) * vectors
test_embeddings = test_mask.unsqueeze(2) * vec
train_embeddings = train_embeddings.reshape(train_embeddings.shape[0], -1).detach().numpy()
test_embeddings = test_embeddings.reshape(test_embeddings.shape[0], -1).detach().numpy()
pooled_train_vec = pooling(train_mask, vectors)
pooled_test_vec = pooling(test_mask, vec)

In [ ]:

inputs = layers.Input(shape=(60*768,))
x2 = layers.Dense(1000, activation="relu")(inputs)
# x2 = layers.Dropout(0.2)(x2)
logits = layers.Dense(100)(x2)
outputs = layers.Softmax()(logits)
model = keras.Model(inputs=inputs, outputs=outputs)
embedding_model = keras.Model(inputs=inputs, outputs=logits)
d = {lab : idx for idx, lab in enumerate(list(set(intent + it)))}
train_lab = np.array([d[i] for i in intent])
test_lab = np.array([d[i] for i in it])
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    train_embeddings, train_lab, batch_size=50, epochs=12, validation_data=(test_embeddings, test_lab)
)